In [28]:
import pandas as pd

In [29]:
df = pd.read_csv("FINAL_SPECIES_RAW.csv")
df.drop("Unnamed: 0", axis=1)

,Sample_ID,Epigenetic_deviation,BMI,Group,Race_1.0,Race_2.0,Race_3.0,Race_4.0,Sex_1.0,Sex_2.0,...,s__uncultured_Allobaculum_sp.,s__Thiobacillus_unclassified,s__Paracoccus_sp._G200V,s__Methylophilus_unclassified,s__SM2D12_unclassified,s__Paracoccus_limosus,s__Candidatus_Arthromitus_unclassified,s__Rothia_sp._MYI-52,s__Candidatus_Soleaferrea_unclassified,s__uncultured_Micrococcus_sp.
0,80004,1.582379,17.545506,Worse,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,80001,-17.008019,23.871528,Better,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,80020,-7.461716,25.778835,Worse,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,80007,-12.790208,27.038133,Better,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,80019,-7.160825,23.262468,Worse,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,43327,-6.386774,29.299593,Worse,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
307,47170,-6.246294,20.115458,Worse,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
308,5703,-12.695643,19.223376,Better,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
309,14654,-12.169833,29.927159,Better,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
import pandas as pd
import numpy as np
from scipy.stats import gmean

# Placeholder DataFrame for actual data (replace with the actual df)
# Assuming df is already loaded with the necessary data
# df = pd.read_csv('path_to_file.csv')

# Extract the portion of the dataframe to be CLR transformed (samples as rows, features as columns)
df_species = df.iloc[:, 15:]

# Function to calculate detection limit and impute zeros with pseudo values

def impute_pseudo_values(df):
    for index, row in df.iterrows():
        #print(index)
        # Step 1: Calculate the total sum for the row
        total_sum = row[1:].sum()
        
        if total_sum > 0:
            # Step 2: Calculate the detection limit
            detection_limit = 1 / total_sum
            
            # Step 3: Define the range for the uniform distribution
            lower_bound = 0.1 * detection_limit
            upper_bound = detection_limit

            # Step 4: Apply pseudo values where there are zeros
            df.iloc[index] = row.apply(
                lambda x: np.random.uniform(lower_bound, upper_bound) if x == 0 else x
            )
    
    return df

# Apply the function to your DataFrame
df_imputed = impute_pseudo_values(df_species)
df1 = df_imputed

def clr(df):
    
    # Assuming the first column is non-numeric (e.g., sample IDs)
    numeric_cols = df.columns[1:]
    clr_values = []
    for index, row in df.iterrows():
        # Extract numeric values
        numeric_row = row[numeric_cols].astype(float)
        
        # Ensure all values are positive
        if (numeric_row <= 0).any():
            raise ValueError(f"Non-positive values found in row {index}.")
        
        # Compute geometric mean
        geom_mean = gmean(numeric_row)
        
        # Compute CLR-transformed values
        clr_row = np.log(numeric_row / geom_mean)
        
        # Append CLR values with index
        clr_values.append(clr_row)
    
    # Create a DataFrame from CLR values
    clr_df = pd.DataFrame(clr_values, columns=numeric_cols)
    
    # If you want to include the non-numeric identifier column
    clr_df.insert(0, df.columns[0], df.iloc[:, 0])
    
    return clr_df

clr_df = clr(df1)
df2 = clr_df

# Add back the first 15 columns to the transformed data
final_df = pd.concat([df.iloc[:, :15], df2], axis=1)

# Save the result to a CSV file
output_file = "/local/data/tws720/FINAL_SPECIES_CLR.csv"
final_df.to_csv(output_file, index=False)
